<a href="https://colab.research.google.com/github/Stravanni/Basi_di_dati/blob/main/05_SQL_sqlite_GRROUP_BY_HAGING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduzione a SQL
@author: giovanni.simonini@unimore.it

## To run: 
- to run a cell: SHIFT + ENTER
  
## Schema Università

S (<u>Matr</u>,SNome,Citta,ACorso)

D(<u>CD</u>,CNome,Citta)

C(<u>CC</u>,CNome,CD)
- FOREIGN KEY (CD) REFERENCES D(CD)

E(<u>Matr,CC</u>,DATA,VOTO)
- FOREIGN KEY (Matr) REFERENCES S(Matr)
- FOREIGN KEY (CC) REFERENCES C(CC)
 

In [3]:
import pandas as pd
from sqlalchemy import create_engine

In [4]:
engine = create_engine('sqlite://', echo=False)

## Create the the tables

In [5]:
q = '''
CREATE TABLE S (
    Matr VARCHAR(45),
    SNome VARCHAR(45),
    Citta VARCHAR(45),
    ACorso INT,
    PRIMARY KEY (Matr)
);
'''
engine.execute(q)

q = '''
CREATE TABLE D(
 CD VARCHAR(45),
 CNome VARCHAR(45),
 Citta VARCHAR(45),
 PRIMARY KEY (CD)
);
'''
engine.execute(q)

q = '''
CREATE TABLE C(
 CC VARCHAR(45),
 CNome VARCHAR(45),
 CD VARCHAR(45),
 PRIMARY KEY (CC),
 FOREIGN KEY (CD) REFERENCES D(CD)
);
'''
engine.execute(q)

q = '''
CREATE TABLE E(
 Matr VARCHAR(45),
 CC VARCHAR(45),
 DATA DATE,
 VOTO INT,
 PRIMARY KEY (Matr, CC),
 FOREIGN KEY (Matr) REFERENCES S(Matr),
 FOREIGN KEY (CC) REFERENCES C(CC)
);
'''
engine.execute(q)


q = '''
INSERT INTO S (Matr, SNome, Citta, ACorso)
VALUES
('M1','Lucia Quaranta','SA',1),
('M2','Giacomo Tedesco','PA',2),
('M3','Carla Longo','MO',1),
('M4','Ugo Rossi','MO',1),
('M5','Valeria Neri','MO',2),
('M6','Giuseppe Verdi','BO',1),
('M7','Maria Rossi',null,1);
'''
engine.execute(q)


q = '''
INSERT INTO D (CD, CNome, Citta)
VALUES
('D1','Paolo Rossi','MO'),
('D2','Maria Pastore','BO'),
('D3','Paola Caboni','FI');
'''
engine.execute(q)


q = '''
INSERT INTO C (CC,CNome, CD)
VALUES
('C1','Fisica 1','D1'),
('C2','Analisi Matematica 1','D2'),
('C3','Fisica 2','D1'),
('C4','Analisi Matematica 2','D3');
'''
engine.execute(q)


q = '''
INSERT INTO E (Matr,CC,Data,Voto)
VALUES
('M1','C1','1995-06-29',24),
('M1','C2','1996-08-09',33),
('M1','C3','1996-03-12',30),
('M2','C1','1995-06-29',28),
('M2','C2','1996-07-07',24),
('M3','C2','1996-07-07',27),
('M3','C3','1996-11-11',25),
('M4','C3','1996-11-11',33),
('M6','C2','1996-01-02',28),
('M7','C1','1995-06-29',24),
('M7','C2','1996-04-11',26),
('M7','C3','1996-06-23',27);
'''
engine.execute(q)

# Query the DB

In [6]:
# TEMPLATE QUERY
q = '''
SELECT *
FROM S,C,D,E
'''

res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df.head()

,Matr,SNome,Citta,ACorso,CC,CNome,CD,CD,CNome,Citta,Matr,CC,DATA,VOTO
0,M1,Lucia Quaranta,SA,1,C1,Fisica 1,D1,D1,Paolo Rossi,MO,M1,C1,1995-06-29,24
1,M1,Lucia Quaranta,SA,1,C1,Fisica 1,D1,D1,Paolo Rossi,MO,M1,C2,1996-08-09,33
2,M1,Lucia Quaranta,SA,1,C1,Fisica 1,D1,D1,Paolo Rossi,MO,M1,C3,1996-03-12,30
3,M1,Lucia Quaranta,SA,1,C1,Fisica 1,D1,D1,Paolo Rossi,MO,M2,C1,1995-06-29,28
4,M1,Lucia Quaranta,SA,1,C1,Fisica 1,D1,D1,Paolo Rossi,MO,M2,C2,1996-07-07,24


# GROUP BY

In una istruzione SELECT e' possibile formare 
dei gruppi di tuple che hanno lo stesso valore 
di specificati attributi, tramite la clausola
- `GROUP BY`


SINTASSI:
```
SELECT [DISTINCT|ALL] <lista-SELECT> FROM <lista-FROM>
[WHERE <condizione>]
[GROUP BY <lista-group> ]
```


**IMPORTANTE**

Il risultato della SELECT e' un unico record per ciascun gruppo

In [10]:
q='''
SELECT Matr, MAX(Voto)
FROM E
GROUP BY Matr
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df.head()

,Matr,MAX(Voto)
0,M1,33
1,M2,28
2,M3,27
3,M4,33
4,M6,28


In [11]:
# “Voto massimo e minimo ottenuto per ogni studente, escludendo il corso C1”
q='''
SELECT Matr, MAX(Voto), MIN(Voto) 
FROM E
WHERE CC <> 'C1'
GROUP BY Matr
'''

res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df.head()

,Matr,MAX(Voto),MIN(Voto)
0,M1,33,30
1,M2,24,24
2,M3,27,25
3,M4,33,33
4,M6,28,28


In [12]:
# Codice e nome di un corso, e relativo numero di esami sostenuti
q='''
SELECT C.CC, C.CNome, COUNT(*) as num_esami
FROM E,C
WHERE E.CC=C.CC
GROUP BY C.CC, C.CNome
'''

res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df.head()

,CC,CNome,num_esami
0,C1,Fisica 1,3
1,C2,Analisi Matematica 1,5
2,C3,Fisica 2,4


# HAVING
In presenza di una clausola WHERE, il raggruppamento riporta solo i valori che soddisfano la condizione della WHERE

In [13]:
# Numero esami e somma dei voti per ogni studente, escludendo il corso C2
q='''
SELECT Matr, COUNT(*), SUM(Voto)
FROM E
WHERE CC <> 'C2'
GROUP BY Matr
'''

res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df.head()

# ma non restituisce alcuna informazione sulla 
# matricola M6, in quanto nessuna tupla di E
# corrispondente a M6 soddisfa la condizione

,Matr,COUNT(*),SUM(Voto)
0,M1,2,54
1,M2,1,28
2,M3,1,25
3,M4,1,33
4,M7,2,51


In [18]:
# Dobbiamo usare GROUP BY ALL per considerare anche i NULL
q='''
SELECT Matr,COUNT(*),SUM(Voto) 
FROM E
WHERE CC <> 'C2'
GROUP BY ALL Matr
'''

# SQLite non supporta GROUP BY ALL

# res = engine.execute(q)
# df = pd.DataFrame(res.fetchall())
# df.columns = res.keys()
# df.head()

,Matr,COUNT(*),SUM(Voto)
0,M1,2,54
1,M2,1,28
2,M3,1,25
3,M4,1,33
4,M7,2,51


Se volessimo gli studenti con una media maggiore di 28?
  
**NON POSSIAMO FARE QUANTO SEGUE:**

In [19]:
q='''
SELECT Matr,AVG(VOTO) 
FROM E
WHERE VOTO > 28
GROUP BY Matr
'''

res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df.head()

,Matr,AVG(VOTO)
0,M1,31.5
1,M4,33.0


### COS'E' LA CLAUSOLA `HAVING`
La clausola HAVING e' l’equivalente dellaclausola WHERE
applicata a gruppi di tuple: 

ogni gruppo costruito tramite GROUP BY 
fa parte del risultato dell’interrogazione 
se e solo se 
il predicato specificato nella clausola HAVING risulta soddisfatto.


**SINTASSI:**
```
[GROUP BY <lista-group> ]
[HAVING <condizione>]
```

In [20]:
# Selezionare studente e media, degli studenti con media maggiore di 28
q='''
SELECT Matr, AVG(VOTO) 
FROM E
GROUP BY Matr
HAVING AVG(VOTO) > 28
'''

res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df.head()

,Matr,AVG(VOTO)
0,M1,29.0
1,M4,33.0


In [23]:
# Media dei voti per ogni esame sostenuto da piu' di due studenti
q='''
SELECT CC, AVG(Voto)
FROM E
GROUP BY CC
HAVING COUNT(*) > 2
'''

res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df.head()

,CC,AVG(Voto)
0,C1,25.333333
1,C2,27.600000
2,C3,28.750000


In [24]:
# Corsi per cui ci sono almeno due studenti con lo stesso voto
q='''
SELECT CC, Voto
FROM E
GROUP BY CC, Voto
HAVING COUNT(Matr) >= 2
'''

res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df.head()

,CC,VOTO
0,C1,24


In [25]:
q='''
SELECT *
FROM E
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df.head()

,Matr,CC,DATA,VOTO
0,M1,C1,1995-06-29,24
1,M1,C2,1996-08-09,33
2,M1,C3,1996-03-12,30
3,M2,C1,1995-06-29,28
4,M2,C2,1996-07-07,24


In [26]:
q='''
SELECT E.Matr,COUNT(*) as num, AVG(Voto)
FROM E join S on E.Matr=S.Matr
WHERE CC <> 'C2'
GROUP BY E.Matr
ORDER BY AVG(Voto) DESC
'''

res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df.head()

,Matr,num,AVG(Voto)
0,M4,1,33.0
1,M2,1,28.0
2,M1,2,27.0
3,M7,2,25.5
4,M3,1,25.0


In [27]:
# Provare aggiungendo S.SNome alla GROUP BY

q='''
SELECT E.Matr, COUNT(*) as num, AVG(Voto)
FROM E join S on E.Matr=S.Matr
WHERE CC <> 'C2'
GROUP BY E.Matr, S.SNome
ORDER BY E.Matr
'''

res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df.head()

,Matr,num,AVG(Voto)
0,M1,2,27.0
1,M2,1,28.0
2,M3,1,25.0
3,M4,1,33.0
4,M7,2,25.5


In [28]:
# Rimuovere di nuovo S.SNome alla GROUP BY
# Aggiungere SELECT anche S.SNome

q='''
SELECT E.Matr, S.SNome, COUNT(*) as num, AVG(Voto)
FROM E join S on E.Matr=S.Matr
WHERE CC <> 'C2'
GROUP BY E.Matr, S.SNome
ORDER BY E.Matr
'''

res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df.head()

,Matr,SNome,num,AVG(Voto)
0,M1,Lucia Quaranta,2,27.0
1,M2,Giacomo Tedesco,1,28.0
2,M3,Carla Longo,1,25.0
3,M4,Ugo Rossi,1,33.0
4,M7,Maria Rossi,2,25.5


# Divisione

In [30]:
# Studenti che hanno sostenuto tutti gli esami relativi a corsi del docente D1
q='''
SELECT *
FROM S
WHERE NOT EXISTS
  (SELECT*
   FROM C
   WHERE CD='D1' AND 
   NOT EXISTS
      (SELECT*
       FROM E
       WHERE E.Matr=S.Matr 
       AND E.CC=C.CC))
'''

res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df.head()

,Matr,SNome,Citta,ACorso
0,M1,Lucia Quaranta,SA,1
1,M7,Maria Rossi,None,1


In [31]:
#“Studenti che hanno sostenuto tutti gli esami sostenuti dallo studente M7”
# riformulata come
#“Studenti per i quali non esiste alcun esame sostenuto da M7 che essi non hanno sostenuto”
q='''
  SELECT *
  FROM S
  WHERE Matr <> 'M7'
  AND NOT EXISTS
    (SELECT *
     FROM E E1 WHERE Matr='M7'
     AND NOT EXISTS
      (SELECT *
       FROM E E2
       WHERE E2.Matr=S.Matr 
       AND E2.CC=E1.CC))
'''

res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df.head()

,Matr,SNome,Citta,ACorso
0,M1,Lucia Quaranta,SA,1
